TL;DR 
Alpha Vantage has been disqualified from further consideration due to: the required endpoint being a premium endpoint.

In [2]:
from decouple import config

In [3]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [36]:
import pytz
from dataclasses import dataclass
from typing import Literal
from urllib.parse import urlencode
from datetime import datetime
from decimal import Decimal
import requests

ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

def transform_alpha_vantage_result(timestamp_str, result):
    tz = pytz.timezone('UTC')
    utc = pytz.utc
    timestamp_formats = [
        '%Y-%m-%d %H:%M:%S',
        '%Y-%m-%d',
    ]
    timestamp = None
    for fmt in timestamp_formats:
        try:
            timestamp = tz.localize(datetime.strptime(timestamp_str, fmt)).astimezone(utc)
            break
        except ValueError:
            continue

    if timestamp is None:
        raise ValueError(
            f"Invalid timestamp format: '{timestamp_str}'. "
            f"Tried formats: {timestamp_formats}."
        )

    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),
        'number_of_trades': None,
        'volume': Decimal(result['5. volume']),
        'volume_weighted_average': None,
        'time': timestamp,
    }




@dataclass
class AlphaVantageAPIClient:
    symbol: str = "BTC"
    market: str = "USD"
    function: Literal["DIGITAL_CURRENCY_DAILY", "DIGITAL_CURRENCY_WEEKLY", "DIGITAL_CURRENCY_MONTHLY", "CRYPTO_INTRADAY"] = "CRYPTO_INTRADAY"
    interval: Literal["1min", "5min", "15min", "30min", "60min"] = "1min"
    api_key: str = ""

    def get_api_key(self):
        return self.api_key or ALPHA_VANTAGE_API_KEY

    def get_headers(self):
        api_key = self.get_api_key()
        return {}

    def get_params(self):
        return {
            "apikey": self.get_api_key(),
            "symbol": self.symbol,
            "market": self.market,
            "interval": self.interval,
            "function": self.function,    
        }
    
    def generate_url(self, pass_auth=False):
        path = "/query"
        url = f"https://www.alphavantage.co{path}"
        params = self.get_params()
        encoded_params = urlencode(params)
        url = f"{url}?{encoded_params}"
        if pass_auth:
            api_key = self.get_api_key()
            url += f"&api_key={api_key}"
        return url

    def fetch_data(self):
        headers = self.get_headers()
        url = self.generate_url()
        response = requests.get(url, headers=headers)
        response.raise_for_status() # not 200/201
        return response.json()

    def get_stock_data(self):
        data = self.fetch_data()

        dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
        results = data[dataset_key]
        dataset = []
        try:
            for timestamp_str in results.keys():
                dataset.append(
                    transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
                )
            return dataset
        except AttributeError:
            return data



In [37]:
stock_api_client = AlphaVantageAPIClient()
dataset = stock_api_client.get_stock_data()
dataset

{'Information': 'Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints'}